In [3]:
# !pip install openpyxl

  Using cached openpyxl-3.1.5-py2.py3-none-any.whl.metadata (2.5 kB)
  Using cached et_xmlfile-2.0.0-py3-none-any.whl.metadata (2.7 kB)
Using cached openpyxl-3.1.5-py2.py3-none-any.whl (250 kB)
Using cached et_xmlfile-2.0.0-py3-none-any.whl (18 kB)


In [5]:
import pandas as pd
df = pd.read_excel('buffet_qna.xlsx')
df.head()

,Section,Questions,Answers
0,Investing Approach,### Have you ever bought a company where the n...,The best buys have been when the numbers almos...
1,Investing Approach,### What is it that really piques your interes...,We're so limited now because we can only go in...
2,Investing Approach,### What is your investment process?,* In the past some things were cheap enough ...
3,Investing Approach,### What's your acquisition criteria? What has...,We look for people who have a passion for thei...
4,Investing Approach,### What's your acquisition strategy? How do y...,We'd like to keep buying businesses like the e...


In [8]:
df_new = df.copy()
df_new['others_ans'] = None
df_new['wb_ans'] = None
df_new

,Section,Questions,Answers,others_ans,wb_ans
0,Investing Approach,### Have you ever bought a company where the n...,The best buys have been when the numbers almos...,None,None
1,Investing Approach,### What is it that really piques your interes...,We're so limited now because we can only go in...,None,None
2,Investing Approach,### What is your investment process?,* In the past some things were cheap enough ...,None,None
3,Investing Approach,### What's your acquisition criteria? What has...,We look for people who have a passion for thei...,None,None
4,Investing Approach,### What's your acquisition strategy? How do y...,We'd like to keep buying businesses like the e...,None,None
...,...,...,...,...,...
434,The Big Picture,"### This was an opportunity, but in thick of t...",Buffett: He has seen some times when undervalu...,None,None
435,The Big Picture,### Is there a way to create synergies between...,CM: Solar solutions are coming because they ar...,None,None
436,The Big Picture,### Rail business – does our country need high...,"WB: By its nature, it is non-economic when com...",None,None
437,The Big Picture,### What would be impact on portfolio of a Chi...,WB: Fire following SFO quake was major cause o...,None,None


In [ ]:
import pandas as pd
import re

patterns = [
    r'(\[CM:([^\[\]]*(?:\[[^\[\]]*\][^\[\]]*)*)\])',                    # [CM: ...]
    r'(\[Cm:([^\[\]]*(?:\[[^\[\]]*\][^\[\]]*)*)\])',
    r'(CM:([^\[\]]*(?:\[[^\[\]]*\][^\[\]]*)*)(?=WB:|\]))',               # CM: ... (up to WB: or end of string)
    r'(CM:([^\[\]]*(?:\[[^\[\]]*\][^\[\]]*)*?)\s*(?=B:|\]))',               # CM: ... (up to B: or end of string)          
    r'(\[Munger:([^\[\]]*(?:\[[^\[\]]*\][^\[\]]*)*)\])',                    # [Munger: ...]
    r'(Munger:([^\[\]]*(?:\[[^\[\]]*\][^\[\]]*)*)(?=Buffett:|\]))',               # CM: ... (up to WB: or end of string)
    r'(Munger:([^\[\]]*(?:\[[^\[\]]*\][^\[\]]*)*)(?=B:|\]))'
]

# Iterate through each row
for idx, row in df_new.iterrows():
    answer = row['Answers']
    extracted_parts = []

    # Extract matching parts
    for pattern in patterns:
        matches = re.findall(pattern, answer, re.DOTALL)  # Capture multiple instances
        # print(matches)
        extracted_parts.extend(matches)
        #     print(extracted_parts)
    # print(extracted_parts)
    unique_extracted = []
    seen = set()

    for ind in range(len(extracted_parts)):
        part = extracted_parts[ind]
        if part not in seen:
            unique_extracted.append(part)
            seen.add(part)
            
    # Remove extracted text from Answer
    for extracted in seen:
        if len(extracted) > 1:
            for item in extracted:  # Iterate through each item in extracted
                if item in answer:  # Check if the item is in the answer
                    answer = answer.replace(item, "").strip()  # Replace and strip only if it's present
        else:
            # print(extracted)
            answer = answer.replace(extracted, "").strip()

    # Update dataframe
    extracted_parts_flattened = []

    for part in extracted_parts:
        if isinstance(part, tuple):  # If the part is a tuple
            extracted_parts_flattened.append(" ".join(part))  # Join the tuple's elements and add to the list
        else:
            extracted_parts_flattened.append(part)  # If it's not a tuple, just append as is

    # Now join all parts into a single string
    df_new.at[idx, 'others_ans'] = " ".join(extracted_parts_flattened).strip()
    df_new.at[idx, 'wb_ans'] = answer  # Store cleaned answer
df_new.to_csv("buffett_qna_intermediate.csv")

In [137]:
import pandas as pd

df_new = pd.read_csv('buffett_qna_intermediate.csv')

df_edited = df_new[['Section', 'Questions', 'others_ans', 'wb_ans']]

# Create an empty list to store combined answers and indicators
combined_answers = []
indicators = []

# Iterate through each row and append answers and indicator to the lists
for idx, row in df_edited.iterrows():
    # Add 'others_ans' and mark it with 0 in the indicator
    combined_answers.append(row['others_ans'])
    indicators.append(0)
    
    # Add 'wb_ans' and mark it with 1 in the indicator
    combined_answers.append(row['wb_ans'])
    indicators.append(1)

# Create the new 'Answers' and 'indicator' columns
df_new_combined = pd.DataFrame({
    'Section': df_new['Section'].repeat(2).reset_index(drop=True),
    'Questions': df_new['Questions'].repeat(2).reset_index(drop=True),
    'Answers': combined_answers,
    'Buffett_statement': indicators
})
df_new_combined = df_new_combined[df_new_combined['Answers'].notna()]
# Display the new DataFrame
df_new_combined.head()
df_new_combined.to_csv("buffett_qna_cleaned.csv")